<a href="https://colab.research.google.com/github/gvelec/AppliedDataScience/blob/master/CapsFinal_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#FINAL CAPSTONE PROJECT: "Finding the Best House"

> In the next cells, I propose a code in python according to the instructions for this final peer-graded assignment.

##Introduction
A family plans to buy a house and ask for a 'New Construction'. When the realtor asks them for their needs, the customers said:

> We need to find a 'Unifamily' Property with sale 'Price' under $400000, 3 or more 'Beds', 2 or more 'Baths' in Orlando, FL. The 'Property' needs to be near the most count of Universities and Colleges in Orlando, FL. Also, the best location for the 'Orlando Science High School' and our 'Office in Down Town'.

##Method
* In the first place, I need to find a 'Unifamily' properties list, available for sale with 3 or more 'Beds', 2 or more 'Baths' in Orlando, FL using a "Realty API". When I get it, the list needs to be filtered to choose properties with a price under $400000. For that, I going to review the   Properties' DataFrame with "Pandas".

* In the second step, according to the 'Mean Distance' from 'High School' and 'Work' to each 'Property' the DataFrame needs to be clustering with "K-means". Also, I select the 'Best Cluster' and find a second properties list.

* In third place, I analyze the 'Venues' around each 'Property', finding the Universities and Colleges using the "Foursquare API". The end of this process is a third property list with most count of 'Venues', which they will be selected to look for a deal.

* In all the processes I visualize the result lists with "Folium" maps.

###Note:
> I use the data sets in CSV files to save each step results in my "Google Drive".

##Table of Contents
1. Getting and Explore Properties Data Set

2. Filtering and Clustering the Properties Data Set

3. Getting Colleges and Universities Data Set

4. Selecting and Saving the Final Results

##Libraries Required

In [ ]:
# Loading the Libraries Required

# Library to have access to my Google Drive
from google.colab import drive

# Library to handle data in a vectorized manner
import numpy as np

# Library to data analsysis
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Library to handle JSON files
import json 

#!conda install -c conda-forge geopy --yes # This line was used before have Foursquare fully tested
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Library to handle requests
import requests 

# Matplotlib and associated plotting modules
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# Library to Data Clustering
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # This line was used before have Foursquare fully tested
import folium # map rendering library

##1.- Getting and Explore Properties Data Set
> Properties Data Set form Realty Data API

###Getting Properties Data Set

In [ ]:
# Getting Properties Data Set form Realty Data API 
url = "https://realtor.p.rapidapi.com/properties/v2/list-for-sale"

bd = "3"              # Beds minimun required
so = "relevance"      # How to sort data required
pt = "single_family"  # Property Type required
bt = "2"              # Baths minimun required
rd = "5"              # Radius in miles from City Coords
nc = "true"           # Check for New Constructions
ct = "Orlando"        # City required
lt = "200"            # Maximun data set size
of = "0"              # Parameter required to this request
st = "FL"             # State required

querystring = {"beds_min":bd,"sort":so,"prop_type":pt,"baths_min":bt,"radius":rd,"is_new_construction":nc,"city":ct,"limit":lt,"offset":of,"state_code":st}

headers = {
    'x-rapidapi-host': "realtor.p.rapidapi.com",
    'x-rapidapi-key': "4abb019623msh314794cbf5f538dp19f134jsna539c97c3bbf"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

if response.ok:
  print('Data Requested Sussecfully')
else:
  print("API don't responds, Data Request Fail")

Data Requested Sussecfully


In [ ]:
# Getting the full data from the response to json
data = response.text
jsondata = json.loads(data)
datasize = len(jsondata['properties'])
print('{} properties had been found.'.format(datasize))

200 properties had been found.


In [ ]:
# Extracting the data from jsondata
data_df = pd.DataFrame(index=['Address','Beds','Baths','Price','Rank','New Construction'])
m = len(jsondata['properties'])
for n in range(m):
  address = jsondata['properties'][n]['address']
  baths = jsondata['properties'][n]['baths']
  beds = jsondata['properties'][n]['beds']
  price = jsondata['properties'][n]['price']
  rank = jsondata['properties'][n]['rank']
  new_cons = jsondata['properties'][n]['client_display_flags']['is_new_construction']
  data_df[n]=[address,beds,baths,price,rank,new_cons]
data_1 = data_df.T
print('This is a first DataFrame (Note: Have a Dict with the Address information.)\n')
data_1.head()

This is a first DataFrame (Note: Have a Dict with the Address information.)



,Address,Beds,Baths,Price,Rank,New Construction
0,"{'city': 'Orlando', 'line': '9250 Neher St', '...",5,5,725230,1,True
1,"{'line': '3105 Eagle Hammock Cir.', 'city': 'K...",4,3,317990,2,True
2,"{'line': '12048 Alder Branch Loop', 'city': 'O...",3,3,223900,3,True
3,"{'line': '8904 Tavistock Lakes Boulevard', 'ci...",4,4,502990,4,True
4,"{'line': '5600 Barletta Drive', 'city': 'Saint...",4,2,364990,5,True


####Formating Properties Data Set

In [ ]:
# Extarcting the Address information to a new Address DataFrame
address_df = pd.DataFrame(index=['line','city','state_code','postal_code','lat','lon'])

for n in range(m):
  line = data_1['Address'][n]['line']
  city = data_1['Address'][n]['city']
  state_code = data_1['Address'][n]['state_code']
  postal_code = data_1['Address'][n]['postal_code']
  lat = data_1['Address'][n]['lat']
  lon = data_1['Address'][n]['lon']
  address_df[n]=[line,city,state_code,postal_code,lat,lon]
address_1 = address_df.T
print('This is a New DataFrame with the Address information.\n')
address_1.head()

This is a New DataFrame with the Address information.



,line,city,state_code,postal_code,lat,lon
0,9250 Neher St,Orlando,FL,32827,28.3684,-81.2511
1,3105 Eagle Hammock Cir.,Kissimmee,FL,34743,28.3407,-81.3392
2,12048 Alder Branch Loop,Orlando,FL,32824,28.3962,-81.3474
3,8904 Tavistock Lakes Boulevard,Orlando,FL,32827,28.3721,-81.2548
4,5600 Barletta Drive,Saint Cloud,FL,34771,28.3336,-81.2173


In [ ]:
# Creating a pivot DataFrame with Addresses
x = pd.DataFrame()
x['Address'] = address_1['line']+'. '+address_1['city']+','+address_1['state_code']+' '+address_1['postal_code']
x['lat'] = address_1['lat']
x['lon'] = address_1['lon']
print('This is a Pivot DataFrame with the Addresses and coords.\n')
x.head()

This is a Pivot DataFrame with the Addresses and coords.



,Address,lat,lon
0,"9250 Neher St. Orlando,FL 32827",28.3684,-81.2511
1,"3105 Eagle Hammock Cir.. Kissimmee,FL 34743",28.3407,-81.3392
2,"12048 Alder Branch Loop. Orlando,FL 32824",28.3962,-81.3474
3,"8904 Tavistock Lakes Boulevard. Orlando,FL 32827",28.3721,-81.2548
4,"5600 Barletta Drive. Saint Cloud,FL 34771",28.3336,-81.2173


In [ ]:
# Adding the Addresses and Coords to the Fist DataFrame
data_1['Address']=x
data_1['lat']=x['lat']
data_1['lon']=x['lon']
print('This is a first DataFrame (Note: With the Addresses and Coords information)\n')
data_1.head()

This is a first DataFrame (Note: With the Addresses and Coords information)



,Address,Beds,Baths,Price,Rank,New Construction,lat,lon
0,"9250 Neher St. Orlando,FL 32827",5,5,725230,1,True,28.3684,-81.2511
1,"3105 Eagle Hammock Cir.. Kissimmee,FL 34743",4,3,317990,2,True,28.3407,-81.3392
2,"12048 Alder Branch Loop. Orlando,FL 32824",3,3,223900,3,True,28.3962,-81.3474
3,"8904 Tavistock Lakes Boulevard. Orlando,FL 32827",4,4,502990,4,True,28.3721,-81.2548
4,"5600 Barletta Drive. Saint Cloud,FL 34771",4,2,364990,5,True,28.3336,-81.2173


####Describing Properties Data Set

In [ ]:
# Describing the Data Set 'Data_1'
print('All the Properties are New Constructions, In Orlando Metro Area, 3+ Beds, 2+ Baths\n') 
data_1.describe(include='all')

All the Properties are New Constructions, In Orlando Metro Area, 3+ Beds, 2+ Baths



,Address,Beds,Baths,Price,Rank,New Construction,lat,lon
count,199,200,200,200,200,200,200.0000,200.0000
unique,71,4,7,163,43,1,72.0000,72.0000
top,"10914 History Avenue. Orlando,FL 32832",4,3,339990,22,True,28.4277,-81.2186
freq,17,93,85,5,5,200,17.0000,17.0000


In [ ]:
# Describing Data Set 'Data_1' Stats
data_1_central=pd.DataFrame(index=['Beds','Baths','Price','Rank','lat','lon'])
data_1_central['mean']=data_1[['Beds','Baths','Price','Rank','lat','lon']].mean()
data_1_central['std']=data_1[['Beds','Baths','Price','Rank','lat','lon']].std()
data_1_central['max']=data_1[['Beds','Baths','Price','Rank','lat','lon']].max()
data_1_central['min']=data_1[['Beds','Baths','Price','Rank','lat','lon']].min()
print('This are the stats for the first properties data set')
data_1_central

This are the stats for the first properties data set


,mean,std,max,min
Beds,3.770000,0.720901,6.000000e+00,3.000000
Baths,3.150000,1.011243,9.000000e+00,2.000000
Price,517496.930000,582609.929503,7.000000e+06,199000.000000
Rank,20.950000,12.210363,4.300000e+01,1.000000
lat,28.415146,0.086984,2.866546e+01,28.289717
lon,-81.302181,0.094818,-8.118553e+01,-81.545086


###Proccesing Properties Data Set
####Slicing Data by Normal Distribution Data Set Dispersion Stats
![Dispersion Stats](https://github.com/gvelec/AppliedDataScience/blob/gvelec-images/dist_slice.png?raw=true)

In [ ]:
# Filtering Data Set 'Data_1' by Dispersion Value Stats with 'Price' under $400000
data_1_sliced=pd.DataFrame(index=['Beds','Baths','Price','Rank','lat','lon'])
data_1_sliced=data_1[data_1['Price']<=(data_1_central['mean']['Price']-data_1_central['std']['Price']/2)] 

max = 400000
n = 2
x = 0
while max >= x:
  x = data_1[data_1['Price']<=(data_1_central['mean']['Price']-data_1_central['std']['Price']/n)]['Price'].max()
  n = n+1

data_1_sliced=data_1[data_1['Price']<=(data_1_central['mean']['Price']-data_1_central['std']['Price']/(n-2))]

print("Properties Filtered {}/{} for 'Price' values bellow $400000 \n Sliced With mean-std/{}, max = ${}, and mean = ${:.2f}\n".format(data_1_sliced['Price'].count(),
                                                                                                                                 data_1['Price'].count(),(n-2),
                                                                                                                                 data_1_sliced['Price'].max(),
                                                                                                                                 data_1_sliced['Price'].mean()))
data_1_sliced.describe()

Properties Filtered 110/200 for 'Price' values bellow $400000 
 Sliced With mean-std/5, max = $399990, and mean = $334409.81



,Address,Beds,Baths,Price,Rank,New Construction,lat,lon
count,110,110,110,110,110,110,110.0000,110.0000
unique,36,4,3,81,42,1,36.0000,37.0000
top,"10914 History Avenue. Orlando,FL 32832",3,3,339990,10,True,28.4277,-81.2186
freq,12,56,57,4,5,110,12.0000,12.0000


In [ ]:
# Describing Data Set 'Data_1_sliced' Stats
data_1_stats=pd.DataFrame(index=['Beds','Baths','Price','Rank','lat','lon'])
data_1_stats['mean']=data_1_sliced[['Beds','Baths','Price','Rank','lat','lon']].mean()
data_1_stats['std']=data_1_sliced[['Beds','Baths','Price','Rank','lat','lon']].std()
data_1_stats['max']=data_1_sliced[['Beds','Baths','Price','Rank','lat','lon']].max()
data_1_stats['min']=data_1_sliced[['Beds','Baths','Price','Rank','lat','lon']].min()
print('This properties data set is the first step result, showing the stats for 1/{} std under the mean.\n'.format(n-2))
data_1_stats

This properties data set is the first step result, showing the stats for 1/5 std under the mean.



,mean,std,max,min
Beds,3.545455,0.615345,6.000000,3.000000
Baths,2.627273,0.588250,4.000000,2.000000
Price,334409.809091,35150.045279,399990.000000,199000.000000
Rank,19.872727,12.257265,43.000000,1.000000
lat,28.411053,0.096318,28.665457,28.289717
lon,-81.292137,0.086024,-81.185532,-81.522500


###Visualizing Properties Data Set on the Map

In [ ]:
address = 'Orlando, FL'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Orlando, FL are 28.5421109, -81.3790304.


In [ ]:
# create map of Orlando using latitude and longitude values
map_orlando = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers for all Postal Codes in DataFrae to map
for lat, lng, price, rank in zip(data_1['lat'], data_1['lon'], data_1['Price'], data_1['Rank']):
    label = '{}, {}'.format(price, rank)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_orlando)  

# add markers to map for Postal Codes for Borough that contain the word 'Toronto' in their names.
for lat, lng, price, rank in zip(data_1_sliced['lat'], data_1_sliced['lon'], data_1_sliced['Price'], data_1_sliced['Rank']):
    label = '{}, {}'.format(price, rank)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF86cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_orlando)

map_orlando

##2.- Filtering and Clustering the Properties Data Set
> Clustering Properties Data Set By Location

###Finding the Geometric Medium Distance From Properties to High School and Work

In [ ]:
# Finding the coords for the Work Place
work_address = 'Corner Lake Middle School'
geolocator = Nominatim(user_agent="to_explorer")
work_location = geolocator.geocode(work_address)
work_latitude = work_location.latitude
work_longitude = work_location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(work_address, work_latitude, work_longitude))

The geograpical coordinate of Corner Lake Middle School are 28.564345799999998, -81.13005542911478.


In [ ]:
# Finding the coords for the Kids' High School
high_school_address = '2600 Technology Dr, Orlando, FL 32804'
geolocator = Nominatim(user_agent="to_explorer")
high_location = geolocator.geocode(high_school_address)
high_latitude = high_location.latitude
high_longitude = high_location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(high_school_address, high_latitude, high_longitude))

The geograpical coordinate of 2600 Technology Dr, Orlando, FL 32804 are 28.55562497366266, -81.41223784197597.


In [ ]:
# Adding a Data Set Column with the Mean Distance from Work and High School to each Property
properties_to_work = ((data_1_sliced['lat']-work_latitude)**2+(data_1_sliced['lon']-work_longitude)**2)**0.5
properties_to_high = ((data_1_sliced['lat']-high_latitude)**2+(data_1_sliced['lon']-high_longitude)**2)**0.5
mean_distance = ((properties_to_work)**2+(properties_to_work)**2)**0.5
data_1_sliced['Mean Dist'] = (mean_distance/mean_distance.max()).apply(lambda x: round(x, 2))
data_1_sliced.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Address,Beds,Baths,Price,Rank,New Construction,lat,lon,Mean Dist
0,"1618 Lake Sims Parkway. Ocoee,FL 34761",4,3,308990,1,True,28.6051,-81.5193,0.99
1,"1618 Lake Sims Parkway. Ocoee,FL 34761",5,3,316990,2,True,28.6051,-81.5193,0.99
2,"1618 Lake Sims Parkway. Ocoee,FL 34761",5,3,321990,3,True,28.6051,-81.5193,0.99
3,"1618 Lake Sims Parkway. Ocoee,FL 34761",3,2,293990,4,True,28.6051,-81.5193,0.99
6,"3105 Eagle Hammock Cir.. Kissimmee,FL 34743",3,3,306990,7,True,28.3407,-81.3392,0.78


###Clustering this Data Set

####Finding the Clusters

In [ ]:
# set number of clusters
kclusters = 5

c=['Address','Beds','Baths','Price','Rank','New Construction','lat','lon']
data_1_sliced_clustering = data_1_sliced.drop(columns=c)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(data_1_sliced_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 4, 4, 4, 3, 0, 3], dtype=int32)

In [ ]:
# add clustering labels to each property
data_1_sliced['Cluster Labels']=kmeans.labels_
col_list=data_1_sliced.columns.tolist()
new_list=list(range(len(col_list)))
for n in list(range(len(col_list))):
  new_list[n]=col_list[n-1]
new_list
data_1_sliced=data_1_sliced[new_list]

data_1_sliced.head() # check the last columns!

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Cluster Labels,Address,Beds,Baths,Price,Rank,New Construction,lat,lon,Mean Dist
0,2,"1618 Lake Sims Parkway. Ocoee,FL 34761",4,3,308990,1,True,28.6051,-81.5193,0.99
1,2,"1618 Lake Sims Parkway. Ocoee,FL 34761",5,3,316990,2,True,28.6051,-81.5193,0.99
2,2,"1618 Lake Sims Parkway. Ocoee,FL 34761",5,3,321990,3,True,28.6051,-81.5193,0.99
3,2,"1618 Lake Sims Parkway. Ocoee,FL 34761",3,2,293990,4,True,28.6051,-81.5193,0.99
6,4,"3105 Eagle Hammock Cir.. Kissimmee,FL 34743",3,3,306990,7,True,28.3407,-81.3392,0.78


In [ ]:
# Cleaning Data Set Clustered
#
# filling 'NaN' values with '0'
print('Before cleaning\n','There are {} columns with "NaN" values in Data\n'.format(data_1_sliced.isnull().any().sum()))
data_1_sliced=data_1_sliced.fillna(0)
print('After cleaning\n', 'There are {} columns with "NaN" values in Data\n'.format(data_1_sliced.isnull().any().sum()))
# setting integer data in 'Cluster Labels' 
data_1_sliced['Cluster Labels']=data_1_sliced['Cluster Labels'].astype(int)
print('Data Set cleaned have {} Properties\n'.format(data_1_sliced.shape[0]))

Before cleaning
 There are 0 columns with "NaN" values in Data

After cleaning
 There are 0 columns with "NaN" values in Data

Data Set cleaned have 110 Properties



####Visualizing the Cluster Data Set on a Map

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(data_1_sliced['lat'], data_1_sliced['lon'], data_1_sliced['Address'], data_1_sliced['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

###Selecting the Best Cluster by Mean Distance

####Describing each Cluster

In [ ]:
# Showing Cluster 0 Estructure
print('Cluster 0 Stats\n')
cluster0 = data_1_sliced[data_1_sliced['Cluster Labels']==0]
cluster0.describe()

Cluster 0 Stats



,Cluster Labels,Beds,Baths,Price,Rank,lat,lon,Mean Dist
count,29.0,29.000000,29.000000,29.000000,29.00000,29.000000,29.000000,29.000000
mean,0.0,3.689655,2.517241,328953.448276,24.00000,28.359721,-81.289798,0.701034
std,0.0,0.541390,0.687682,35708.988442,12.75035,0.073460,0.065202,0.014478
min,0.0,3.000000,2.000000,199000.000000,3.00000,28.289717,-81.409783,0.670000
25%,0.0,3.000000,2.000000,314990.000000,14.00000,28.308548,-81.346810,0.690000
50%,0.0,4.000000,2.000000,333995.000000,24.00000,28.333585,-81.297520,0.700000
75%,0.0,4.000000,3.000000,349990.000000,36.00000,28.393053,-81.223995,0.710000
max,0.0,5.000000,4.000000,386995.000000,43.00000,28.634829,-81.206569,0.730000


In [ ]:
# Showing Cluster 1 Estructure
print('Cluster 1 Stats\n')
data_1_sliced[data_1_sliced['Cluster Labels']==1].describe()

Cluster 1 Stats



,Cluster Labels,Beds,Baths,Price,Rank,lat,lon,Mean Dist
count,31.0,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,1.0,3.516129,2.677419,336901.290323,18.161290,28.470762,-81.246496,0.397097
std,0.0,0.724383,0.599283,31866.892301,11.602002,0.042081,0.033393,0.043680
min,1.0,3.000000,2.000000,256000.000000,4.000000,28.427700,-81.318177,0.300000
25%,1.0,3.000000,2.000000,318990.000000,8.500000,28.427700,-81.286881,0.350000
50%,1.0,3.000000,3.000000,339990.000000,15.000000,28.479141,-81.235528,0.410000
75%,1.0,4.000000,3.000000,358990.000000,26.500000,28.495640,-81.218600,0.430000
max,1.0,6.000000,4.000000,395990.000000,43.000000,28.559794,-81.214789,0.480000


In [ ]:
# Showing Cluster 2 Estructure
print('Cluster 2 Stats\n')
data_1_sliced[data_1_sliced['Cluster Labels']==2].describe()

Cluster 2 Stats



,Cluster Labels,Beds,Baths,Price,Rank,lat,lon,Mean Dist
count,8.0,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
mean,2.0,3.875000,3.000000,349865.000000,14.000000,28.602855,-81.519447,0.990000
std,0.0,0.834523,0.534522,44309.584581,14.392458,0.011372,0.003808,0.013093
min,2.0,3.000000,2.000000,293990.000000,1.000000,28.575140,-81.522500,0.960000
25%,2.0,3.000000,3.000000,314990.000000,2.750000,28.605100,-81.522500,0.990000
50%,2.0,4.000000,3.000000,342740.000000,9.000000,28.605100,-81.519300,0.990000
75%,2.0,4.250000,3.000000,396615.000000,21.500000,28.609100,-81.519300,1.000000
max,2.0,5.000000,4.000000,399990.000000,38.000000,28.609100,-81.510876,1.000000


In [ ]:
# Showing Cluster 3 Estructure
print('Cluster 3 Stats\n')
data_1_sliced[data_1_sliced['Cluster Labels']==3].describe()

Cluster 3 Stats



,Cluster Labels,Beds,Baths,Price,Rank,lat,lon,Mean Dist
count,29.0,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000
mean,3.0,3.379310,2.620690,339546.206897,19.896552,28.355460,-81.253233,0.631379
std,0.0,0.493804,0.493804,34008.485644,11.693542,0.041610,0.047288,0.031929
min,3.0,3.000000,2.000000,278990.000000,2.000000,28.328749,-81.365438,0.580000
25%,3.0,3.000000,2.000000,309000.000000,11.000000,28.333583,-81.267612,0.590000
50%,3.0,3.000000,3.000000,339990.000000,16.000000,28.341523,-81.254840,0.650000
75%,3.0,4.000000,3.000000,372990.000000,30.000000,28.354238,-81.231623,0.660000
max,3.0,4.000000,3.000000,396990.000000,42.000000,28.469219,-81.185532,0.660000


In [ ]:
# Showing Cluster 4 Estructure
print('Cluster 4 Stats\n')
data_1_sliced[data_1_sliced['Cluster Labels']==4].describe()

Cluster 4 Stats



,Cluster Labels,Beds,Baths,Price,Rank,lat,lon,Mean Dist
count,13.0,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
mean,4.0,3.461538,2.538462,319671.461538,18.307692,28.389166,-81.353098,0.785385
std,0.0,0.518875,0.518875,36309.375572,11.600177,0.122645,0.041951,0.024703
min,4.0,3.000000,2.000000,267990.000000,4.000000,28.335548,-81.447361,0.770000
25%,4.0,3.000000,2.000000,291990.000000,8.000000,28.335548,-81.339196,0.770000
50%,4.0,3.000000,3.000000,308829.000000,15.000000,28.340735,-81.339196,0.780000
75%,4.0,4.000000,3.000000,348990.000000,25.000000,28.340735,-81.330225,0.780000
max,4.0,4.000000,3.000000,375000.000000,38.000000,28.665457,-81.330225,0.840000


####Selecting the best Cluster

In [ ]:
# Selecting the best Cluster by Mean Distance
cluster = 0
mean_dist = 1.0
for n in range(5):
  x = data_1_sliced[data_1_sliced['Cluster Labels']==n]['Mean Dist'].mean()
  if x <= mean_dist:
    mean_dist = x
    cluster = n
print('The Best Cluster is #{} with a Mean Distance of {:.2f}\n'.format(cluster,mean_dist))

The Best Cluster is #1 with a Mean Distance of 0.40



In [ ]:
data_2 = data_1_sliced[data_1_sliced['Cluster Labels']==cluster]
data_2

,Cluster Labels,Address,Beds,Baths,Price,Rank,New Construction,lat,lon,Mean Dist
26,1,"5009 Santa Rosa Dr. Orlando,FL 32807",3,2,289000,27,True,28.555226,-81.318177,0.48
27,1,"5015 Santa Rosa Dr. Orlando,FL 32807",3,2,289000,28,True,28.555280,-81.317985,0.48
35,1,"10914 History Avenue. Orlando,FL 32832",4,3,395990,36,True,28.427700,-81.218600,0.41
42,1,"10914 History Avenue. Orlando,FL 32832",4,3,373990,43,True,28.427700,-81.218600,0.41
49,1,"10914 History Avenue. Orlando,FL 32832",3,2,362990,7,True,28.427700,-81.218600,0.41
54,1,"5973 Wooden Pine Drive. Orlando,FL 32829",4,3,358990,12,True,28.479141,-81.235528,0.34
55,1,"5973 Wooden Pine Drive. Orlando,FL 32829",3,2,331990,13,True,28.479141,-81.235528,0.34
56,1,"5973 Wooden Pine Drive. Orlando,FL 32829",4,2,339990,14,True,28.479141,-81.235528,0.34
57,1,"5973 Wooden Pine Drive. Orlando,FL 32829",3,2,322990,15,True,28.479141,-81.235528,0.34
58,1,"5973 Wooden Pine Drive. Orlando,FL 32829",4,3,378990,16,True,28.479141,-81.235528,0.34


####Visualizing Selected Cluster on a Map

In [ ]:
# create map Cluster Selected
cluster_lat = data_2['lat'].mean()
cluster_lon = data_2['lon'].mean()

map_cluster = folium.Map(location=[cluster_lat, cluster_lon], zoom_start=11)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(data_2['lat'], data_2['lon'], data_2['Price'], data_2['Address']):
    label = folium.Popup('$' + str(poi) + ' "Address" ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_cluster)

label = folium.Popup(str('Work'), parse_html=True)
folium.CircleMarker(
        [work_latitude, work_longitude],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_cluster)

label = folium.Popup(str('High School'), parse_html=True)
folium.CircleMarker(
        [high_latitude, high_longitude],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_cluster)

map_cluster

##3.- Getting Colleges and Universities Data Set
> Venues Data Set from Foursquare

###Setting the Foursquare Credentials

In [ ]:
CLIENT_ID = 'WKCLES5O0RMEY2MH45E5S4M4MNAYHBCC5QPCAYXTDTULQ2NC' # your Foursquare ID
CLIENT_SECRET = 'SVUKFIJOYB4HPNKBXH5YIWFKDPV2HQ2ABBIDAM23S4DPTQQ0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

###Creating our Request Get Venues Funtion

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=12000, LIMIT=200, categoryId='4d4b7105d754a06372d81259'):
    venues_list=[]
    i = 0
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        propertyID=data_2.index.values.tolist()[i] # getting the index in data_2 DataFrame for each property
        venues_list.append([(
            propertyID,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        i=i+1
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'propertyID',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']    
    return(nearby_venues)

###Getting the Venues and Creating a Dictionary

In [ ]:
# creating the venues dictionary
venues={}
properties = data_2.index.values.tolist()
i=0
for n in ['University', 'College']:
  # getting the group DataFrames
  x = getNearbyVenues(names=n,latitudes=data_2['lat'],longitudes=data_2['lon'])
  #
  print('For {} have {} Venues'.format(n,x.shape[0]))
  venues[n]=x # saving the vanues in a dictionary
  i=i+1
#

For University have 371 Venues
For College have 284 Venues


##4.- Selecting and Saving the Final Results
> Final Properties Data Set.

###Filtering Properties Data Set by Venues Count

In [ ]:
# Filtering and Counting the Venues by CategoryID (Colleges and Universities)
venues_college = venues['College'].loc[venues['College']['Venue Category'].isin(['University',
                                                                                 'College & University'])].groupby('propertyID').count()
venues_university = venues['University'].loc[venues['University']['Venue Category'].isin(['University',
                                                                                          'College & University'])].groupby('propertyID').count()
# Finding the max venues count by categoryID in Venues Data set
venues_total = round((venues_college + venues_university)/2)
max_venues = venues_total.max()['Venue']

# Getting the list of propertiesID with the max Venues count
properties_max = venues_total[venues_total['Venue']==max_venues].index.tolist()

# Filtering from Poperties Data Set by PropertyID
data_3 = data_2.loc[properties_max]

data_3

,Cluster Labels,Address,Beds,Baths,Price,Rank,New Construction,lat,lon,Mean Dist
26,1,"5009 Santa Rosa Dr. Orlando,FL 32807",3,2,289000,27,True,28.555226,-81.318177,0.48
27,1,"5015 Santa Rosa Dr. Orlando,FL 32807",3,2,289000,28,True,28.555280,-81.317985,0.48


###Formating the Result Data Set

In [ ]:
# Getting the Colleges & Universities List Near to the Best Properties
venues_final = venues['University'].loc[venues['University']['Venue Category'].isin(['University','College & University'])]
venues_final_list = venues_final[venues_final['propertyID']==data_3.index.values[0]]['Venue'].tolist()

# Filtering Properties Data set
data_top = data_3.reset_index()
data_top.drop(columns=['Cluster Labels','lat','lon','Rank','New Construction','Mean Dist'], inplace=True)

# Getting the Final Result Properties List
results_final = pd.DataFrame()
results_final[['Address','Beds','Baths','Price']]=data_top[['Address','Beds','Baths','Price']]

print('Properties List from the Best Cluster\n Max number of Universities and Colleges = {}'.format(max_venues))
print(' {}\n'.format(venues_final_list))
results_final

Properties List from the Best Cluster
 Max number of Universities and Colleges = 4.0
 ['Rollins College', 'Full Sail University', 'UCF Downtown Campus', 'National University - Orlando, Florida']



,Address,Beds,Baths,Price
0,"5009 Santa Rosa Dr. Orlando,FL 32807",3,2,289000
1,"5015 Santa Rosa Dr. Orlando,FL 32807",3,2,289000


###Visualizing the Results on a Map 

In [ ]:
# create map Final Properties Selected
final_lat = data_3['lat'].mean()
final_lon = data_3['lon'].mean()

map_final = folium.Map(location=[final_lat, final_lon], zoom_start=12)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(data_3['lat'], data_3['lon'], data_3['Price'], data_3['Address']):
    label = folium.Popup('$' + str(poi) + ' "Address" ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_final)

label = folium.Popup(str('Work'), parse_html=True)
folium.CircleMarker(
        [work_latitude, work_longitude],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_final)

label = folium.Popup(str('High School'), parse_html=True)
folium.CircleMarker(
        [high_latitude, high_longitude],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_final)

for lat, lon, poi, cluster in zip(venues_final['Venue Latitude'], venues_final['Venue Longitude'], venues_final['Venue'], venues_final['Venue Category']):
    label = folium.Popup(str(poi) + ' ' + str(cluster), parse_html=True)
    folium.Marker(
        [lat, lon],
        icon=folium.Icon(color='orange', icon='info-sign'),
        popup=label,
        tooltip='Click Me').add_to(map_final)

map_final



##NOT ESCENTIAL: This code give access to Google Drive

In [ ]:
# getting access to drive as local repository
from google.colab import drive
ROOT = "/content/drive/"
drive.mount(ROOT)
#

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Next code permit me save results to Google Drive

In [ ]:
# saving the new results like CSV file in Google Drive.
data_1.to_csv('final_data1.csv')
!cp final_data1.csv "drive/My Drive/DataScience"
data_2.to_csv('final_data2.csv')
!cp final_data2.csv "drive/My Drive/DataScience"
data_3.to_csv('final_data3.csv')
!cp final_data3.csv "drive/My Drive/DataScience"
results_final.to_csv('final_results.csv')
!cp final_results.csv "drive/My Drive/DataScience"

# [If you can't see the MAPS follow this link](https://github.com/gvelec/AppliedDataScience/commit/ae49be79b3bc240be2e5aeecd308f8233f895b14#commitcomment-41538439)